In [1]:
from aide_design.play import*
from aide_design import floc_model as floc
from aide_design import cdc_functions as cdc
from aide_design.unit_process_design.prefab import lfom_prefab_functional as lfom
from pytexit import py2tex
import math


# 1 L/s Plants in Parallel 

# CHANCEUX
## Priya Aggarwal, Sung Min Kim, Felix Yang



### Monroe's comments 

* Add the team name under the title. **fixed**
* Add the names of the students who are writing this report under the team name. **fixed**
* See if you can figure out how to access PACl viscosity. I believe it is defined in the floc model python file. PACl stock solution viscosity is quite a bit higher than water. **fixed**
* Use SI units in your analysis **fixed**
* Document your code. Explain your solution approach before jumping in to Python.
* The picture you intended to show wasn't available to me. Include it as a zipped file next time.
* below you do math in the markdown text box. Use python to do math!
* Discuss pro/con of standardization and modular design vs different designs for each flow rate. 
* Discuss changes that a community would need to make to add capacity by purchasing another 1 L/s plant. Note the impact of standardization on the changes they would need to make.
* Always strive to create the general solution. In this case let the target flow rate be a variable and use that to set the number of plants that will be working in parallel. This will make your code much more useful and will actually improve the documentation of the code because you will no longer be using random constants. Instead you will be using variables with clear names.
* Once you determine which pieces of the plant change with flow rate, then you can focus on creating the design algorithms for those components.
* I'm eager to see what you develop



### Pros/Cons
* Pros for standardization : Mass production is going to be preferred if this technology becomes more widely used.
* Cons against standardization: Not every situation will be the same.

* Pros for different designs : Can be made for each and every situation. We also don't have a very big range. 
* Cons against different designs: Tedious and busy work. 

AguaClara, which began building plants in Honduras in 2005, India in 2013, and most recently Nicaragua in 2017, has provided gravity powered water treatment systems to thousands of people in rural communities. However, there were still populations that could not be targeted due to AguaClara technology only being scaled up from 6 L/s. For towns and rural areas with populations smaller than a thousand, sustainable water treatment technology was still out of their grasp.

Recently a one liter per second (1 LPS) plant was developed based on traditional AguaClara technology, to bring sustainable water treatment to towns with populations of at 300 people per plant. 

The first plant that was ever fabricated was sent to Cuatro Comunidades in Honduras and is currently operating without the filter attachment, also known as EStaRS. Enclosed Stacked Rapid Sand Filter is the last step in the 1LPS plant processes before chlorination and completes the 4 step water treatment process of: flocculation, sedimentation, filtration, and chlorination.
Having water treatment plants for smaller plants would increase AguaClara’s reach and allow it to help more people. Despite being in its initial stages, the demand for this technology is increasing. Three 1 LPS were recently ordered for a town that did not have water treatment, compared to Cuatro Communidades where there already was a running water treatment plant. However, the implementation of treatment plants is a problem that has not been solved. 

This project has stemmed from the possibility of implementing AguaClara technologies to be helpful in Puerto Rico’s post hurricane rebuild effort. This goal of this project is to assess whether the portable 1 L/s plant could be a viable option to help rural communities have safe drinking water. The project models a series of 1 L/s plants working in parallel to provide for the community. 

Based on our initial response to this problem below is the solution we are proposing. Sedimentation Tanks should definitely be run in parallel and flow coming into the plant doesn’t have to be split until after the entrance tank or even after the flocculator. This is because one entrance tank and one chemical dosing controller would be more easily manageable than three.
1.	Bigger entrance tank according to flow rate - as opposed to 3 entrance tanks.
2.	Bigger Flocculator - As opposed to 3 flocculators.
3.	More coagulant/CL according to flow
4.	Sed Tanks in Parallel
5.	Recombine flow for Filtration

We show further calculations to prove why these are the best options




In [2]:
Q_Plant=3*u.L/u.s
T = u.Quantity(20,u.degC)
Headloss=10*u.cm 
DesiredCl_Concentration=3*u.mg/u.L
C_stock_PACl=70*u.gram/u.L
NuPaCl = cdc.viscosity_kinematic_pacl(C_stock_PACl,T)
RatioError = 0.1
KMinor = 2
Q_CDC=(Q_plant*DesiredCl_Concentration/C_stock_PACl).to(u.mL/u.s)
#print('The flow rate of coagulant for 3 L/s is', Q_CDC)
#print('The flow rate of coagulant for 3 L/s per hour is ',Q_CDC.to(u.L/u.hour)) ##L/hr

#VA_Plant=3VA_Singular
#A_Plant=3A_Singular
#(pi*D_2^2)/4=(pi*D_1^2)/4
#D_2^2=3*D_1^2
#D_2=(3^.5)*D_1
Q_Singular_Floc=(1/3)*Q_plant
D_Flocculator=3*u.inch

D_new_floc= np.sqrt(3)*D_Flocculator ## This 3 inch is based off of the current 1lPS
#print('The diameter of the flocculator plant is',D_new_floc)

def FlocculatorSize(Q_Plant):
    Diam_Flocculator=(Q_Plant.magnitude**.5)*3*u.inch
    SDR=26
    Pipe_Diam=pipe.ND_SDR_available(Diam_Flocculator,SDR)
    print(Pipe_Diam)
    return(Diam_Flocculator)
FlocculatorSize(Q_Plant)





NameError: name 'Q_plant' is not defined

SPACE CONSTRAINTS: 

SED TANK: Each sedimentation tank is 36 in diameter. The top half is 42.75 inches slanted. 
x component that protrudes out= 42.75*cos(30)-18=19 inches. 

FLOCCULATOR: We want symmetrical piping system coming out of the flocculator so that available head going into the parallel sedimentation tanks will be the same. We will have an asymmetrical exit lauder system coming out of the sedimentation tanks going into the STaRS (diagram). 

Additionally the picture should have the sedimentation tanks in reverse so that the slanted side is closer to the platform. 
![image.png](attachment:image.png)


The horizontal length of the plant was decided to be 21 ft because we wanted 3 feet in between each plant and 3 feet between the sedimentation tanks and the walls. 3 feet was chosen so that there would be sufficient space for the operator to walk between. Since each tank is 3 feet wide in diameter, this meant that there would be four 3 feet spaces between the tanks and between the walls and 3 tanks so total was 7*3=21 ft.

Vertical length was decided to be 21 ft by 6 feet of space reserved for getting behind the plant. In the bottom of the picture there is a platform that will be the same height as the sedimentation tank so about 2 meters so that the plant operator can peer into the sedimenation tank as well as have access to the plate settlers. An additional 7 feet was reserved for the STaRS see calculation below. Lastly about 4 feet was added for the y-axis length of the sedimentation tank and another 4 feet between the flocculator and sedimentation tanks. 


In [ ]:
Sed_Tank_Diameter=36*u.inch
Length_Top_Half=42.75*u.inch
X_Sed_Tank=Length_Top_Half*math.cos(30)
print(X_Sed_Tank.to(u.m))

In [ ]:
Space_Between_Sed=3*u.ft
Spaces=7
Total_Ho_Length=Space_Between_Sed*Spaces
Space_Platform=6*u.ft
Vertical_Length_Sed=4*u.ft
Space_Between_Flocc_Sed=4*u.ft

print('The total horizontal length is',Total_Ho_Length.to(u.m))
Area_Plant=21*u.ft*Total_Ho_Length
Surface_Area_STaRS=(1.67*u.m**2/u.L)*3*u.L
Length_STaRS=Surface_Area_STaRS**.5
Total_Ve_Length=(np.round(Length_STaRS.to(u.ft)))+Space_Platform+Vertical_Length_Sed+Space_Between_Flocc_Sed
print('The total vertical length is',Total_Ve_Length.to(u.m))
print('The area that the STaRS will require is',Surface_Area_STaRS.to(u.m**2)) ##Approximately 7 ftx7ft
print('Total area of the plant is',Area_Plant.to(u.m**2))

In [5]:
def Horizontal(Q):
    Q_Plant=Q.to(u.L/u.s)
    Extra_Space=6*u.ft
    Horizontal_Distance=(2*Q_Plant.magnitude-1)*3*u.ft+Extra_Space #
    return(Horizontal_Distance.to(u.m))
Q_Plant_2=1*u.L/u.s
Horizontal(Q_Plant)

<Quantity(6.400799999999999, 'meter')>

In [9]:
ESTaRS_SA=1*u.m**2/u.L
def Area_ESTaRS(Q):
    if Q.magnitude!=type(integer)
    Q_Plant=Q.to(u.L/u.s)
    Surface_Area_ESTaRS=(ESTaRS_SA.magnitude)*Q_Plant.magnitude
    print('Surface area required for the ESTaR(s) is',Surface_Area_ESTaRS*u.m**2)
    return(Surface_Area_ESTaRS*u.m**2)
Area_ESTaRS(Q_Plant)
        

Surface area required for the ESTaR(s) is 3 meter ** 2


<Quantity(3.0, 'meter ** 2')>

In [ ]:
def Horizontal(Q):
    Q_Plant=Q.to(u.L/u.s)
    Extra_Space=6*u.ft
    Horizontal_Distance=(Q_Plant.magnitude)*3*u.ft+(Q_Plant.magnitude-1)*.1*u.m+Extra_Space #
    return(Horizontal_Distance.to(u.m))

Q_Plant_2=1*u.L/u.s
Horizontal(Q_Plant)